In [1]:
from scipy.misc import *
import numpy as np
import pylab as plb
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.integrate import quad
from scipy.integrate import nquad
from scipy.misc import derivative
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import widgets
from matplotlib.widgets import Slider, Button, RadioButtons, CheckButtons, TextBox # Matplotlib widgets
from astropy import table as Table # For fast and easy reading / writing with tables using numpy library
import pandas as pd
import emcee
import corner
from astropy import table as Table # For fast and easy reading / writing with tables using numpy library
from galpy.potential import MiyamotoNagaiPotential, NFWPotential, RazorThinExponentialDiskPotential, BurkertPotential # GALPY potentials

In [2]:
# In[ ]:

tt=Table.Table.read('coordinates.txt', format='ascii.tab') # importando los datos de las imágenes

#Importando coordenadas de las imágenes deflectadas
theta1=tt['theta1'] 
theta2=tt['theta2']

theta=np.zeros(len(theta1),float)
for i in range(len(theta1)):
    theta[i]=np.sqrt(theta1[i]**2+theta2[i]**2)
    
tt=Table.Table.read('parameters_lens_source.txt', format='ascii.tab') # importando los datos de distancias cosmológicas

#Importando parámetros de la fuente
R=tt['FIT'][0]
H=tt['FIT'][1] 
K=tt['FIT'][2]

FC = np.pi/(180*3600) #conversion factor between arcs and radians
h = H*FC
K = K*FC
r = R*FC

tt=Table.Table.read('alpha.txt', format='ascii.tab') # importando los datos de las imágenes los ángulos en la circunferencia para la fuente circular
#Imprtando los valores de alpha
alpha=tt['alpha']

Beta1 = r*np.cos(alpha)+h
Beta2 = r*np.sin(alpha)+K

tt=Table.Table.read('Cosmological_distances.txt', format='ascii.tab') # importando los datos de distancias cosmológicas
#Importando distancias cosmológicas y Sigma Crítico
D_ds=tt['D_ds'][0] 
D_d=tt['D_d'][0]
D_s=tt['D_s'][0]
SIGMA_CRIT=tt['SIGMA_CRIT'][0]

In [3]:
#Define deflector angle for each gravitational potential
# Burket potential
def alpha1_burket(M_0,r_s):
    GRADPOT1 = np.zeros((len(theta1)), float)
    def POTDEF1(z,TheTa1):
        TheTa = np.sqrt(TheTa1**2+theta2[l]**2)
        R = D_d*TheTa
        Burket_p = BurkertPotential(amp=M_0, a=r_s, normalize=False)
        Sigma = Burket_p.dens(R,z)
        kappa = Sigma/SIGMA_CRIT
        return (4/theta1[l])*TheTa1*kappa/SIGMA_CRIT**2
    for l in range(len(theta1)):
        GRADPOT1[l]= nquad(POTDEF1, [[0, np.inf],[0, theta1[l]]])[0]
    return GRADPOT1*SIGMA_CRIT**2

def alpha2_burket(M_0,r_s):
    GRADPOT1 = np.zeros((len(theta1)), float)
    def POTDEF1(z,TheTa2):
        TheTa = np.sqrt(TheTa2**2+theta1[l]**2)
        R = D_d*TheTa
        Burket_p = BurkertPotential(amp=M_0, a=r_s, normalize=False)
        Sigma = Burket_p.dens(R,z)
        kappa = Sigma/SIGMA_CRIT
        return (4/theta2[l])*TheTa2*kappa/SIGMA_CRIT**2
    for l in range(len(theta1)):
        GRADPOT1[l]= nquad(POTDEF1, [[0, np.inf],[0, theta2[l]]])[0]
    return GRADPOT1*(SIGMA_CRIT**2)

#Miyamoto Nagai potential
def alpha1_MN1(M1,b1):
    GRADPOT1 = np.zeros((len(theta1)), float)
    def POTDEF1(z,TheTa1):
        TheTa = np.sqrt(TheTa1**2+theta2[l]**2)
        R = D_d*TheTa
        MN_Bulge_p= MiyamotoNagaiPotential(amp=M1,a=0,b=b1,normalize=False)
        Sigma = MN_Bulge_p.dens(R,z)
        kappa = Sigma/SIGMA_CRIT
        return (4/theta1[l])*TheTa1*kappa/SIGMA_CRIT**2
    for l in range(len(theta1)):
        GRADPOT1[l]= nquad(POTDEF1, [[0, np.inf],[0, theta1[l]]])[0]
    return GRADPOT1*(SIGMA_CRIT**2)

def alpha2_MN1(M1,b1):
    GRADPOT1 = np.zeros((len(theta1)), float)
    def POTDEF1(z,TheTa2):
        TheTa = np.sqrt(TheTa2**2+theta1[l]**2)
        R = D_d*TheTa
        MN_Bulge_p= MiyamotoNagaiPotential(amp=M1,a=0,b=b1,normalize=False)
        Sigma = MN_Bulge_p.dens(R,z)
        kappa = Sigma/SIGMA_CRIT
        return (4/theta2[l])*TheTa2*kappa/SIGMA_CRIT**2
    for l in range(len(theta1)):
        GRADPOT1[l]= nquad(POTDEF1, [[0, np.inf],[0, theta2[l]]])[0]
    return GRADPOT1*(SIGMA_CRIT**2)


def alpha1_MN(M2,a2,b2):
    GRADPOT1 = np.zeros((len(theta1)), float)
    def POTDEF1(z,TheTa1):
        TheTa = np.sqrt(TheTa1**2+theta2[l]**2)
        R = D_d*TheTa
        MN_Bulge_p= MiyamotoNagaiPotential(amp=M2,a=a2,b=b2,normalize=False)
        Sigma = MN_Bulge_p.dens(R,z)
        kappa = Sigma/SIGMA_CRIT
        return (4/theta1[l])*TheTa1*kappa/SIGMA_CRIT**2
    for l in range(len(theta1)):
        GRADPOT1[l]= nquad(POTDEF1, [[0, np.inf],[0, theta1[l]]])[0]
    return GRADPOT1*(SIGMA_CRIT**2)

def alpha2_MN(M2,a2,b2):
    GRADPOT1 = np.zeros((len(theta1)), float)
    def POTDEF1(z,TheTa2):
        TheTa = np.sqrt(TheTa2**2+theta1[l]**2)
        R = D_d*TheTa
        MN_Bulge_p= MiyamotoNagaiPotential(amp=M2,a=a2,b=b2,normalize=False)
        Sigma = MN_Bulge_p.dens(R,z)
        kappa = Sigma/SIGMA_CRIT
        return (4/theta2[l])*TheTa2*kappa/SIGMA_CRIT**2
    for l in range(len(theta1)):
        GRADPOT1[l]= nquad(POTDEF1, [[0, np.inf],[0, theta2[l]]])[0]
    return GRADPOT1*(SIGMA_CRIT**2)

#NFW potential
def alpha1_NFW(M_0,r_s):
    GRADPOT1 = np.zeros((len(theta1)), float)
    def POTDEF1(z,TheTa1):
        TheTa = np.sqrt(TheTa1**2+theta2[l]**2)
        R = D_d*TheTa
        NFW_p = NFWPotential(amp=M_0, a=r_s, normalize=False)
        Sigma = NFW_p.dens(R,z)
        kappa = Sigma/SIGMA_CRIT
        return (4/theta1[l])*TheTa1*kappa/SIGMA_CRIT**2
    for l in range(len(theta1)):
        GRADPOT1[l]= nquad(POTDEF1, [[0, np.inf],[0, theta1[l]]])[0]
    return GRADPOT1*(SIGMA_CRIT**2)

def alpha2_NFW(M_0,r_s):
    GRADPOT1 = np.zeros((len(theta1)), float)
    def POTDEF1(z,TheTa2):
        TheTa = np.sqrt(TheTa2**2+theta1[l]**2)
        R = D_d*TheTa
        NFW_p = NFWPotential(amp=M_0, a=r_s, normalize=False)
        Sigma = NFW_p.dens(R,z)
        kappa = Sigma/SIGMA_CRIT
        return (4/theta2[l])*TheTa2*kappa/SIGMA_CRIT**2
    for l in range(len(theta1)):
        GRADPOT1[l]= nquad(POTDEF1, [[0, np.inf],[0, theta2[l]]])[0]
    return GRADPOT1*SIGMA_CRIT**2

#Exponential Disk potential
def alpha1_ExpDisk(Sigma_0,h_r):
    GRADPOT1 = np.zeros((len(theta1)), float)
    def POTDEF1(TheTa1,theta):
        TheTa = np.sqrt(TheTa1**2+theta2[l]**2)
        R = D_d*TheTa
        Sigma = Sigma_0*np.exp(-D_d*TheTa/h_r)
        kappa = Sigma/SIGMA_CRIT
        return (2/theta1[l])*TheTa1*kappa/SIGMA_CRIT**2
    for l in range(len(theta1)):
        GRADPOT1[l]= quad(POTDEF1, 0, theta1[l], limit=100, args=(theta[l]))[0]
    return GRADPOT1*SIGMA_CRIT**2

def alpha2_ExpDisk(Sigma_0,h_r):
    GRADPOT1 = np.zeros((len(theta1)), float)
    def POTDEF1(TheTa2,theta):
        TheTa = np.sqrt(TheTa2**2+theta1[l]**2)
        R = D_d*TheTa
        Sigma = Sigma_0*np.exp(-D_d*TheTa/h_r)
        kappa = Sigma/SIGMA_CRIT
        return (2/theta2[l])*TheTa2*kappa/SIGMA_CRIT**2
    for l in range(len(theta1)):
        GRADPOT1[l]= quad(POTDEF1, 0, theta2[l], limit=100, args=(theta[l]))[0]
    return GRADPOT1*SIGMA_CRIT**2

In [4]:
def THETA_1(M1,b1,M2,a2,b2,r_s,M_0):
    THETA1 = np.zeros((len(theta1)), float)
    THETA2 = np.zeros((len(theta1)), float)
    GRADPOT1 = np.zeros((len(theta1)), float)
    GRADPOT2 = np.zeros((len(theta1)), float)
    GRADPOT1_BULGE=alpha1_MN1(M1,b1)
    GRADPOT2_BULGE=alpha2_MN1(M1,b1)
    GRADPOT1_DISK=alpha1_MN(M2,a2,b2)
    GRADPOT2_DISK=alpha2_MN(M2,a2,b2)
    if Halo==1:
        GRADPOT1_HALO=alpha1_NFW(M_0,r_s)
        GRADPOT2_HALO=alpha2_NFW(M_0,r_s)
    if Halo==2:
        GRADPOT1_HALO=alpha1_burket(M_0,r_s)
        GRADPOT2_HALO=alpha2_burket(M_0,r_s)
    for l in range(len(theta1)):
        GRADPOT1[l]=GRADPOT1_BULGE[l]+GRADPOT1_DISK[l]+GRADPOT1_HALO[l]
        GRADPOT2[l]=GRADPOT2_BULGE[l]+GRADPOT2_DISK[l]+GRADPOT2_HALO[l]
    for l in range(len(theta1)):
        THETA1[l] = Beta1[l]+GRADPOT1[l]
        THETA2[l] = Beta2[l]+GRADPOT2[l]
    #Graphics of source and images
    fig = plt.figure()
#    ax = fig.add_subplot(111)
    plt.rcParams['figure.figsize'] =(5,5)
    plb.plot(Beta1/FC, Beta2/FC, 'or')
    plb.plot(theta1/FC, theta2/FC, 'ob')
    plb.plot(THETA1/FC, THETA2/FC, 'og')
    plb.xlim(-2.5,2.5)
    plb.ylim(-2.5,2.5)
    plb.legend(['Source', 'Observational data', 'model values'], loc='upper right', fontsize=15)
 #   ax.set_aspect('equal', adjustable='box')

def THETA_2(M1,b1,Sigma_0,h_r,r_s,M_0):
    THETA1 = np.zeros((len(theta1)), float)
    THETA2 = np.zeros((len(theta1)), float)
    GRADPOT1 = np.zeros((len(theta1)), float)
    GRADPOT2 = np.zeros((len(theta1)), float)
    GRADPOT1_BULGE=alpha1_MN1(M1,b1)
    GRADPOT2_BULGE=alpha2_MN1(M1,b1)
    GRADPOT1_DISK=alpha1_ExpDisk(Sigma_0,h_r)
    GRADPOT2_DISK=alpha2_ExpDisk(Sigma_0,h_r)
    if Halo==1:
        GRADPOT1_HALO=alpha1_NFW(M_0,r_s)
        GRADPOT2_HALO=alpha2_NFW(M_0,r_s)
    if Halo==2:
        GRADPOT1_HALO=alpha1_burket(M_0,r_s)
        GRADPOT2_HALO=alpha2_burket(M_0,r_s)
    for l in range(len(theta1)):
        GRADPOT1[l]=GRADPOT1_BULGE[l]+GRADPOT1_DISK[l]+GRADPOT1_HALO[l]
        GRADPOT2[l]=GRADPOT2_BULGE[l]+GRADPOT2_DISK[l]+GRADPOT2_HALO[l]
    for l in range(len(theta1)):
        THETA1[l] = Beta1[l]+GRADPOT1[l]
        THETA2[l] = Beta2[l]+GRADPOT2[l]
    #Graphics of source and images
    fig = plt.figure()
#    ax = fig.add_subplot(111)
    plt.rcParams['figure.figsize'] =(5,5)
    plb.plot(Beta1/FC, Beta2/FC, 'or')
    plb.plot(theta1/FC, theta2/FC, 'ob')
    plb.plot(THETA1/FC, THETA2/FC, 'og')
    plb.xlim(-2.5,2.5)
    plb.ylim(-2.5,2.5)
    plb.legend(['Source', 'Observational data', 'model values'], loc='upper right', fontsize=15)
 #   ax.set_aspect('equal', adjustable='box')

def THETA_3(M1,a1,b1,M2,a2,b2,r_s,M_0):
    THETA1 = np.zeros((len(theta1)), float)
    THETA2 = np.zeros((len(theta1)), float)
    GRADPOT1 = np.zeros((len(theta1)), float)
    GRADPOT2 = np.zeros((len(theta1)), float)
    GRADPOT1_BULGE=alpha1_MN(M1,a1,b1)
    GRADPOT2_BULGE=alpha2_MN(M1,a1,b1)
    GRADPOT1_DISK=alpha1_MN(M2,a2,b2)
    GRADPOT2_DISK=alpha2_MN(M2,a2,b2)
    if Halo==1:
        GRADPOT1_HALO=alpha1_NFW(M_0,r_s)
        GRADPOT2_HALO=alpha2_NFW(M_0,r_s)
    if Halo==2:
        GRADPOT1_HALO=alpha1_burket(M_0,r_s)
        GRADPOT2_HALO=alpha2_burket(M_0,r_s)
    for l in range(len(theta1)):
        GRADPOT1[l]=GRADPOT1_BULGE[l]+GRADPOT1_DISK[l]+GRADPOT1_HALO[l]
        GRADPOT2[l]=GRADPOT2_BULGE[l]+GRADPOT2_DISK[l]+GRADPOT2_HALO[l]
    for l in range(len(theta1)):
        THETA1[l] = Beta1[l]+GRADPOT1[l]
        THETA2[l] = Beta2[l]+GRADPOT2[l]
    #Graphics of source and images
    fig = plt.figure()
#    ax = fig.add_subplot(111)
    plt.rcParams['figure.figsize'] =(5,5)
    plb.plot(Beta1/FC, Beta2/FC, 'or')
    plb.plot(theta1/FC, theta2/FC, 'ob')
    plb.plot(THETA1/FC, THETA2/FC, 'og')
    plb.xlim(-2.5,2.5)
    plb.ylim(-2.5,2.5)
    plb.legend(['Source', 'Observational data', 'model values'], loc='upper right', fontsize=15)
 #   ax.set_aspect('equal', adjustable='box')

def THETA_4(M1,a1,b1,Sigma_0,h_r,r_s,M_0):
    THETA1 = np.zeros((len(theta1)), float)
    THETA2 = np.zeros((len(theta1)), float)
    GRADPOT1 = np.zeros((len(theta1)), float)
    GRADPOT2 = np.zeros((len(theta1)), float)
    GRADPOT1_BULGE=alpha1_MN(M1,a1,b1)
    GRADPOT2_BULGE=alpha2_MN(M1,a1,b1)
    GRADPOT1_DISK=alpha1_ExpDisk(Sigma_0,h_r)
    GRADPOT2_DISK=alpha2_ExpDisk(Sigma_0,h_r)
    if Halo==1:
        GRADPOT1_HALO=alpha1_NFW(M_0,r_s)
        GRADPOT2_HALO=alpha2_NFW(M_0,r_s)
    if Halo==2:
        GRADPOT1_HALO=alpha1_burket(M_0,r_s)
        GRADPOT2_HALO=alpha2_burket(M_0,r_s)
    for l in range(len(theta1)):
        GRADPOT1[l]=GRADPOT1_BULGE[l]+GRADPOT1_DISK[l]+GRADPOT1_HALO[l]
        GRADPOT2[l]=GRADPOT2_BULGE[l]+GRADPOT2_DISK[l]+GRADPOT2_HALO[l]
    for l in range(len(theta1)):
        THETA1[l] = Beta1[l]+GRADPOT1[l]
        THETA2[l] = Beta2[l]+GRADPOT2[l]
    #Graphics of source and images
    fig = plt.figure()
#    ax = fig.add_subplot(111)
    plt.rcParams['figure.figsize'] =(5,5)
    plb.plot(Beta1/FC, Beta2/FC, 'or')
    plb.plot(theta1/FC, theta2/FC, 'ob')
    plb.plot(THETA1/FC, THETA2/FC, 'og')
    plb.xlim(-2.5,2.5)
    plb.ylim(-2.5,2.5)
    plb.legend(['Source', 'Observational data', 'model values'], loc='upper right', fontsize=15)
 #   ax.set_aspect('equal', adjustable='box')

In [5]:
Bulge=int(input("Enter the number of bulge profile: 1 for Bulge I, 2 for Bulge II"))
Disk=int(input("Enter the number of disk profile: 1 for Disk thin, 2 for Disk thick, 3 for Exponential disk"))
Halo=int(input("Enter the number of Halo profile: 1 for NFW, 2 for Burket"))

#For the user choose the potential of each mass component of the galaxy

chk=[]

if Bulge==1:
    min_b1=0.0
    max_b1=2.0
    min_M1=0.1e10
    max_M1=2.0e10
    chk.append(True)
else:
    chk.append(False)
    
if Bulge==2:
    min_b1=0.5
    max_b1=1.5
    min_M1=1e10
    max_M1=5e10
    min_a1=0.01
    max_a1=0.05
    chk.append(True)
else:
    chk.append(False)

if Disk==3:
    min_Sigma_0=1e8
    max_Sigma_0=1e10
    min_hr=2
    max_hr=30
    chk.append(True)
else:
    chk.append(False)
    
    
if Disk==1:
    min_b2=0.1
    max_b2=1.0
    min_M2=0.5e11
    max_M2=1.5e11
    min_a2=1.0
    max_a2=10
    chk.append(True)
else:
    chk.append(False)
    
if Disk==2:
    min_b2=0.1
    max_b2=15
    min_M2=0.5e11
    max_M2=1.5e11
    min_a2=1
    max_a2=10
    chk.append(True)
else:
    chk.append(False)
        
if Halo==1:
    min_M0=0.1e11
    max_M0=1e12
    min_rs=0.1
    max_rs=60.1
    chk.append(True)
else:
    chk.append(False)
    
if Halo==2:
    min_M0=0.1e6
    max_M0=10e6
    min_rs=2.0
    max_rs=38.0
    chk.append(True)
else:
    chk.append(False)
    
#Number of steps
N=20

if Bulge==1 and Disk==1:
    A=interactive(THETA_1,b1=widgets.FloatSlider(min=min_b1, max=max_b1, step=(max_b1-min_b1)/N, value=(max_b1-min_b1)/2),
        M1=widgets.FloatSlider(min=min_M1, max=max_M1, step=(max_M1-min_M1)/N, value=(max_M1-min_M1)/2),
        a2=widgets.FloatSlider(min=min_a2, max=max_a2, step=(max_a2-min_a2)/N, value=(max_a2-min_a2)/2),
        b2=widgets.FloatSlider(min=min_b2, max=max_b2, step=(max_b2-min_b2)/N, value=(max_b2-min_b2)/2),
        M2=widgets.FloatSlider(min=min_M2, max=max_M2, step=(max_M2-min_M2)/N, value=(max_M2-min_M2)/2),
        M_0=widgets.FloatSlider(min=min_M0, max=max_M0, step=(max_M0-min_M0)/N, value=(max_M0-min_M0)/2),
        r_s=widgets.FloatSlider(min=min_rs, max=max_rs, step=(max_rs-min_rs)/N, value=(max_rs-min_rs)/2));
    display(A)
    button = widgets.Button(description="Finish")
    output = widgets.Output()
    display(button, output)
    def on_button_clicked(b):
        with output:
            b1=A.kwargs['b1']
            M1=A.kwargs['M1']
            a2=A.kwargs['a2']
            M2=A.kwargs['M2']
            b2=A.kwargs['b2']
            r_s=A.kwargs['r_s']
            M_0=A.kwargs['M_0']
            compnts = ['BULGE I', 'BULGE II','EXP.DISC','THIN DISC','THICK DISC','DARK HALO', 'BURKERT HALO']
            if Halo==1:
                masses = [M1,"None","None",M2,"None",M_0,"None"]
                aa = ["None","None","None",a2,"None",r_s,"None"]
            if Halo==2:
                masses = [M1,"None","None",M2,"None","None",M_0]
                aa = ["None","None","None",a2,"None","None",r_s]
            bb = [b1, "None", "None",b2,"None","None","None"]
            init_parameters = Table.Table([compnts,masses, aa,bb,chk], names=('component', 'mass', 'a (kpc)', 'b (kpc)', 'checked'))
            init_parameters.write('init_guess_params.txt', format='ascii.tab', overwrite=True)
            print("--------------Done---------------")
            print(init_parameters)
    button.on_click(on_button_clicked)

if Bulge==1 and Disk==2:
    A=interactive(THETA_1,b1=widgets.FloatSlider(min=min_b1, max=max_b1, step=(max_b1-min_b1)/N, value=(max_b1-min_b1)/2),
        M1=widgets.FloatSlider(min=min_M1, max=max_M1, step=(max_M1-min_M1)/N, value=(max_M1-min_M1)/2),
        a2=widgets.FloatSlider(min=min_a2, max=max_a2, step=(max_a2-min_a2)/N, value=(max_a2-min_a2)/2),
        b2=widgets.FloatSlider(min=min_b2, max=max_b2, step=(max_b2-min_b2)/N, value=(max_b2-min_b2)/2),
        M2=widgets.FloatSlider(min=min_M2, max=max_M2, step=(max_M2-min_M2)/N, value=(max_M2-min_M2)/2),
        M_0=widgets.FloatSlider(min=min_M0, max=max_M0, step=(max_M0-min_M0)/N, value=(max_M0-min_M0)/2),
        r_s=widgets.FloatSlider(min=min_rs, max=max_rs, step=(max_rs-min_rs)/N, value=(max_rs-min_rs)/2));
    display(A)
    button = widgets.Button(description="Finish")
    output = widgets.Output()
    display(button, output)
    def on_button_clicked(b):
        with output:
            b1=A.kwargs['b1']
            M1=A.kwargs['M1']
            a2=A.kwargs['a2']
            M2=A.kwargs['M2']
            b2=A.kwargs['b2']
            r_s=A.kwargs['r_s']
            M_0=A.kwargs['M_0']
            compnts = ['BULGE I', 'BULGE II','EXP.DISC','THIN DISC','THICK DISC','DARK HALO', 'BURKERT HALO']
            if Halo==1:
                masses = [M1,"None","None","None",M2,M_0,"None"]
                aa = ["None","None","None","None",a2,r_s,"None"]
            if Halo==2:
                masses = [M1,"None","None","None",M2,"None",M_0]
                aa = ["None","None","None","None",a2,"None",r_s]
            bb = [b1, "None", "None","None",b2,"None","None"]
            init_parameters = Table.Table([compnts,masses, aa,bb,chk], names=('component', 'mass', 'a (kpc)', 'b (kpc)', 'checked'))
            init_parameters.write('init_guess_params.txt', format='ascii.tab', overwrite=True)
            print("--------------Done---------------")
            print(init_parameters)
    button.on_click(on_button_clicked)

    
if Bulge==1 and Disk==3:
    A=interactive(THETA_2,b1=widgets.FloatSlider(min=min_b1, max=max_b1, step=(max_b1-min_b1)/N, value=(max_b1-min_b1)/2),
        M1=widgets.FloatSlider(min=min_M1, max=max_M1, step=(max_M1-min_M1)/N, value=(max_M1-min_M1)/2),
        Sigma_0=widgets.FloatSlider(min=min_Sigma_0, max=max_Sigma_0, step=(max_Sigma_0-min_Sigma_0)/N, value=(max_Sigma_0-min_Sigma_0)/2),
        h_r=widgets.FloatSlider(min=min_hr, max=max_hr, step=(max_hr-min_hr)/N, value=(max_hr-min_hr)/2),
        M_0=widgets.FloatSlider(min=min_M0, max=max_M0, step=(max_M0-min_M0)/N, value=(max_M0-min_M0)/2),
        r_s=widgets.FloatSlider(min=min_rs, max=max_rs, step=(max_rs-min_rs)/N, value=(max_rs-min_rs)/2));
    display(A)
    button = widgets.Button(description="Finish")
    output = widgets.Output()
    display(button, output)
    def on_button_clicked(b):
        with output:
            b1=A.kwargs['b1']
            M1=A.kwargs['M1']
            h_r=A.kwargs['h_r']
            Sigma_0=A.kwargs['Sigma_0']
            r_s=A.kwargs['r_s']
            M_0=A.kwargs['M_0']
            compnts = ['BULGE I', 'BULGE II','EXP.DISC','THIN DISC','THICK DISC','DARK HALO', 'BURKERT HALO']
            if Halo==1:
                masses = [M1,"None",Sigma_0,"None","None",M_0,"None"]
                aa = ["None","None",h_r,"None","None",r_s,"None"]
            if Halo==2:
                masses = [M1,"None",Sigma_0,"None","None","None",M_0]
                aa = ["None","None",h_r,"None","None","None",r_s]
            bb = [b1, "None", "None","None","None","None","None"]
            init_parameters = Table.Table([compnts,masses, aa,bb,chk], names=('component', 'mass', 'a (kpc)', 'b (kpc)', 'checked'))
            init_parameters.write('init_guess_params.txt', format='ascii.tab', overwrite=True)
            print("--------------Done---------------")
            print(init_parameters)
    button.on_click(on_button_clicked)
     
if Bulge==2 and Disk==1:
    A=interactive(THETA_3,a1=widgets.FloatSlider(min=min_a1, max=max_a1, step=(max_a1-min_a1)/N, value=(max_a1-min_a1)/2),
        b1=widgets.FloatSlider(min=min_b1, max=max_b1, step=(max_b1-min_b1)/N, value=(max_b1-min_b1)/2),
        M1=widgets.FloatSlider(min=min_M1, max=max_M1, step=(max_M1-min_M1)/N, value=(max_M1-min_M1)/2),
        a2=widgets.FloatSlider(min=min_a2, max=max_a2, step=(max_a2-min_a2)/N, value=(max_a2-min_a2)/2),
        b2=widgets.FloatSlider(min=min_b2, max=max_b2, step=(max_b2-min_b2)/N, value=(max_b2-min_b2)/2),
        M2=widgets.FloatSlider(min=min_M2, max=max_M2, step=(max_M2-min_M2)/N, value=(max_M2-min_M2)/2),
        M_0=widgets.FloatSlider(min=min_M0, max=max_M0, step=(max_M0-min_M0)/N, value=(max_M0-min_M0)/2),
        r_s=widgets.FloatSlider(min=min_rs, max=max_rs, step=(max_rs-min_rs)/N, value=(max_rs-min_rs)/2));
    display(A)
    button = widgets.Button(description="Finish")
    output = widgets.Output()
    display(button, output)
    def on_button_clicked(b):
        with output:
            a1=A.kwargs['a1']
            b1=A.kwargs['b1']
            M1=A.kwargs['M1']
            a2=A.kwargs['a2']
            M2=A.kwargs['M2']
            b2=A.kwargs['b2']
            r_s=A.kwargs['r_s']
            M_0=A.kwargs['M_0']
            compnts = ['BULGE I', 'BULGE II','EXP.DISC','THIN DISC','THICK DISC','DARK HALO', 'BURKERT HALO']
            if Halo==1:
                masses = ["None",M1,"None",M2,"None",M_0,"None"]
                aa = ["None",a1,"None",a2,"None",r_s,"None"]
            if Halo==2:
                masses = ["None",M1,"None",M2,"None","None",M_0]
                aa = ["None",a1,"None",a2,"None","None",r_s]
            bb = ["None",b1, "None",b2,"None","None","None"]
            init_parameters = Table.Table([compnts,masses, aa,bb,chk], names=('component', 'mass', 'a (kpc)', 'b (kpc)', 'checked'))
            init_parameters.write('init_guess_params.txt', format='ascii.tab', overwrite=True)
            print("--------------Done---------------")
            print(init_parameters)
    button.on_click(on_button_clicked)
    

if Bulge==2 and Disk==2:
    A=interactive(THETA_3,a1=widgets.FloatSlider(min=min_a1, max=max_a1, step=(max_a1-min_a1)/N, value=(max_a1-min_a1)/2),
        b1=widgets.FloatSlider(min=min_b1, max=max_b1, step=(max_b1-min_b1)/N, value=(max_b1-min_b1)/2),
        M1=widgets.FloatSlider(min=min_M1, max=max_M1, step=(max_M1-min_M1)/N, value=(max_M1-min_M1)/2),
        a2=widgets.FloatSlider(min=min_a2, max=max_a2, step=(max_a2-min_a2)/N, value=(max_a2-min_a2)/2),
        b2=widgets.FloatSlider(min=min_b2, max=max_b2, step=(max_b2-min_b2)/N, value=(max_b2-min_b2)/2),
        M2=widgets.FloatSlider(min=min_M2, max=max_M2, step=(max_M2-min_M2)/N, value=(max_M2-min_M2)/2),
        M_0=widgets.FloatSlider(min=min_M0, max=max_M0, step=(max_M0-min_M0)/N, value=(max_M0-min_M0)/2),
        r_s=widgets.FloatSlider(min=min_rs, max=max_rs, step=(max_rs-min_rs)/N, value=(max_rs-min_rs)/2));
    display(A)
    button = widgets.Button(description="Finish")
    output = widgets.Output()
    display(button, output)
    def on_button_clicked(b):
        with output:
            a1=A.kwargs['a1']
            b1=A.kwargs['b1']
            M1=A.kwargs['M1']
            a2=A.kwargs['a2']
            M2=A.kwargs['M2']
            b2=A.kwargs['b2']
            r_s=A.kwargs['r_s']
            M_0=A.kwargs['M_0']
            compnts = ['BULGE I', 'BULGE II','EXP.DISC','THIN DISC','THICK DISC','DARK HALO', 'BURKERT HALO']
            if Halo==1:
                masses = ["None",M1,"None","None",M2,M_0,"None"]
                aa = ["None",a1,"None","None",a2,r_s,"None"]
            if Halo==2:
                masses = ["None",M1,"None","None",M2,"None",M_0]
                aa = ["None",a1,"None","None",a2,"None",r_s]
            bb = ["None",b1,"None","None",b2,"None","None"]
            init_parameters = Table.Table([compnts,masses, aa,bb,chk], names=('component', 'mass', 'a (kpc)', 'b (kpc)', 'checked'))
            init_parameters.write('init_guess_params.txt', format='ascii.tab', overwrite=True)
            print("--------------Done---------------")
            print(init_parameters)
    button.on_click(on_button_clicked)


if Bulge==2 and Disk==3:
    A=interact(THETA_4,a1=widgets.FloatSlider(min=min_a1, max=max_a1, step=(max_a1-min_a1)/N, value=(max_a1-min_a1)/2),
        b1=widgets.FloatSlider(min=min_b1, max=max_b1, step=(max_b1-min_b1)/N, value=(max_b1-min_b1)/2),
        M1=widgets.FloatSlider(min=min_M1, max=max_M1, step=(max_M1-min_M1)/N, value=(max_M1-min_M1)/2),
        Sigma_0=widgets.FloatSlider(min=min_Sigma_0, max=max_Sigma_0, step=(max_Sigma_0-min_Sigma_0)/N, value=(max_Sigma_0-min_Sigma_0)/2),
        h_r=widgets.FloatSlider(min=min_hr, max=max_hr, step=(max_hr-min_hr)/N, value=(max_hr-min_hr)/2),
        M_0=widgets.FloatSlider(min=min_M0, max=max_M0, step=(max_M0-min_M0)/N, value=(max_M0-min_M0)/2),
        r_s=widgets.FloatSlider(min=min_rs, max=max_rs, step=(max_rs-min_rs)/N, value=(max_rs-min_rs)/2));
    display(A)
    button = widgets.Button(description="Finish")
    output = widgets.Output()
    display(button, output)
    def on_button_clicked(b):
        with output:
            a1=A.kwargs['a1']
            b1=A.kwargs['b1']
            M1=A.kwargs['M1']
            h_r=A.kwargs['h_r']
            Sigma_0=A.kwargs['Sigma_0']
            r_s=A.kwargs['r_s']
            M_0=A.kwargs['M_0']
            compnts = ['BULGE I', 'BULGE II','EXP.DISC','THIN DISC','THICK DISC','DARK HALO', 'BURKERT HALO']
            if Halo==1:
                masses = ["None",M1,Sigma_0,"None","None",M_0,"None"]
                aa = ["None",a1,h_r,"None","None",r_s,"None"]
            if Halo==2:
                masses = ["None",M1,Sigma_0,"None","None","None",M_0]
                aa = ["None",a1,h_r,"None","None","None",r_s]
            bb = ["None",b1,"None","None","None","None","None"]
            init_parameters = Table.Table([compnts,masses, aa,bb,chk], names=('component', 'mass', 'a (kpc)', 'b (kpc)', 'checked'))
            init_parameters.write('init_guess_params.txt', format='ascii.tab', overwrite=True)
            print("--------------Done---------------")
            print(init_parameters)
    button.on_click(on_button_clicked)


Enter the number of bulge profile: 1 for Bulge I, 2 for Bulge II1
Enter the number of disk profile: 1 for Disk thin, 2 for Disk thick, 3 for Exponential disk3
Enter the number of Halo profile: 1 for NFW, 2 for Burket1


interactive(children=(FloatSlider(value=9500000000.0, description='M1', max=20000000000.0, min=1000000000.0, s…

Button(description='Finish', style=ButtonStyle())

Output()